<a href="https://colab.research.google.com/github/cuberisu/2024project/blob/main/llama2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chat with Multiple PDFs Using Llama2 + Pinecone API + LangChain  
### Free LLMs and Embeddings  
- [Llama2](https://huggingface.co/meta-llama/Llama-2-7b-chat-hf): 이 모델에서 사용한, Meta의 오픈 소스 LLM.  
  - 왜 오픈소스를 써야 하나?
    1. 데이터가 Privacy가 필요한 경우 오픈소스 LLM 모델은 로컬 혹은 내부 서버에서 실행할 수 있다.  
    2. LLM을 비즈니스 목적에 맞게 다시 학습을 하거나, 파인튜닝을 통해 새로운 소규모 모델을 운영할 수 있다.  
    3. OpenAI 등에서 제공하는 대형 LLM을 사용하는데 소요되는 비용과 의존성에 따른 잠재적인 리스크를 줄일 수 있다.  
  - [오픈소스 AI 트렌드 2023.04](https://revf.tistory.com/282)  

- [Pinecone](https://docs.pinecone.io/guides/getting-started/overview): API(외부 데이터(PDF)와의 통합을 위한 인터페이스).  
  LangChain을 설치하면 `langchain-core`, `langchain-community`, `langsmith` 등이 함께 설치되어 프로젝트 수행에 필수적인 라이브러리들은 한번에 설치된다. 다만, 최소한의 요구 사항만 충족되는 것이고, 다양한 외부 모델 제공자와 데이터 저장소 등과의 통합을 위해서는 개별적으로 <mark>의존성 설치</mark>가 필요하다. 예를 들면, OpenAI에서 제공하는 LLM을 사용하려면 `langchain-openai` 의존성 라이브러리를 설치해야 한다.  
  우리는 Meta에서 제공하는 오픈 소스 LLM을 이용하므로, 의존성 따위 필요없고 그냥 외부 데이터베이스를 이용하기 위해서 Pinecone API를 이용하기로 한다.  

- [LangChain](https://wikidocs.net/233341)  
  언어 모델을 활용해 다양한 애플리케이션을 개발할 수 있는 프레임워크.  
  - 문맥인식: `LangChain`은 언어 모델을 다양한 문맥 소스와 연결한다. 여기에는 프롬프트 지시사항, 소수의 예시, 응답에 근거한 내용 등이 포함된다. 이를 통해 언어 모델은 제공된 정보를 기반으로 더 정확하고 관련성 높은 답변을 생성할 수 있다.  
  - 추론: 언어 모델은 주어진 문맥을 바탕으로 어떠한 답변을 제공하거나, 어떤 조치를 취해야 할지를 스스로 추론할 수 있다. 이는 언어 모델이 단순히 정보를 재생산하는 것을 넘어서, 주어진 상황을 분석하고 적절한 해결책을 제시할 수 있음을 의미한다.  
  `LangChain`을 활용하면 이러한 기능을 바탕으로 검색 강화 생성(RAG) 어플리케이션 제작, 구조화된 데이터 분석, 챗봇 등을 만들 수 있다.  

[reference link(YouTube)](https://www.youtube.com/watch?v=TcJ_tVSGS4g)  


## Step 01: Install All the Required Packages
일단 이 프로젝트 내에서는 한 번만 설치하면 된다.  
런타임 > 세션 다시 시작 해도 이미 설치됨.  
(런타임은 뭐고 세션은 뭐고 런타임 유형은 또 뭔지! ㅋㅋㅋㅋ)  
다만, 런타임 유형을 새로 변경하면 (CPU를 GPU로 바꾸는 등) 패키지는 새로 깔아야 한다.  
충격적인 것은 colab 내에서  
CPU와 GPU가 각각 사용하는 패키지 버전이 다르다는 점이다!  
따라서 이 모델은 GPU를 사용해야 한다.  
Colab은 무료 GPU를 지원하는 Linux 기반 가상머신(VM)이다.

In [1]:
! pip install langchain
! pip install pinecone-client
! pip install sentence_transformers
! pip install pdf2image
! pip install pypdf
! pip install xformers
! pip install bitsandbytes accelerate transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.8/276.8 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.9/215.9 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 71.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# ! pip install accelerate
# ! pip install -i https://pypi.org/simple/ bitsandbytes

## pip version, upgrade, and torch test

In [ ]:
# !pip --version

pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)


In [ ]:
# !pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 16.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [ ]:
# !torch
''' Why?
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.2.1+cu121 requires torch==2.2.1, but you have torch 2.2.2 which is incompatible.
torchtext 0.17.1 requires torch==2.2.1, but you have torch 2.2.2 which is incompatible.
torchvision 0.17.1+cu121 requires torch==2.2.1, but you have torch 2.2.2 which is incompatible.
Successfully installed torch-2.2.2 xformers-0.0.25.post1
'''

/bin/bash: line 1: torch: command not found


## Step 02: Import All the Required Libraries

In [ ]:
from langchain.document_loaders import PyPDFDirectoryLoader  # PDF Load
from langchain.text_splitter import RecursiveCharacterTextSplitter  # Text Split
from langchain.embeddings import HuggingFaceEmbeddings  # Embedding
from langchain.vectorstores import Pinecone  # Use Vectors
import pinecone
from pdf2image import convert_from_path
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
import os
import sys

In [ ]:
from langchain import HuggingFacePipeline, PromptTemplate
from langchain.chains import RetrievalQA  # QA model과 연결

## Step 03: Load the PDF Files
여러 개의 PDF를 특정 로컬 경로에 집어넣어 준비하는 과정.  
먼저 `./pdfs` 폴더 만들고, `gdown` 명령어를 이용해 구글 드라이브에 저장된 PDF 논문을 id를 이용해 다운로드한다.

In [ ]:
!mkdir pdfs # 이것도 세션 다시 시작해도 여전히 남아 있다.

In [ ]:
!gdown 1hPQlXrX8FbaYaLypxTmeVOFNitbBMlEE -O pdfs/yolov7paper.pdf
!gdown 1vILwiv6nS2wI3chxNabMgry3qnV67TxM -O pdfs/rachelgreecv.pdf

Downloading...
From: https://drive.google.com/uc?id=1hPQlXrX8FbaYaLypxTmeVOFNitbBMlEE
To: /content/pdfs/yolov7paper.pdf
100% 2.27M/2.27M [00:00<00:00, 206MB/s]
Downloading...
From: https://drive.google.com/uc?id=1vILwiv6nS2wI3chxNabMgry3qnV67TxM
To: /content/pdfs/rachelgreecv.pdf
100% 271k/271k [00:00<00:00, 130MB/s]


## Step 04: Extract the text from the PDF's
- 사용한 PDF Loader: `PyPDFDirectoryLoader` in `langchain_community.document_loaders`  
  특정 폴더의 모든 PDF 문서 로드! [방법](https://wikidocs.net/232110)  
- 사용한 함수: `load()`

In [ ]:
loader = PyPDFDirectoryLoader('pdfs')
data = loader.load()
# data

## Step 05: Split the Extracted Data into Text Chunks
- <mark>Chunk</mark>: 큰 문서를 더 작은 청크(작은 덩어리)로 분할하면 모델 포함의 최대 토큰 입력 한도를 유지하는 데 도움이 된다. LLM에는 청크 크기에 대한 성능 지침이 있다. 사용 중인 모든 모델에 가장 적합한 청크 크기를 설정해야 한다.  
  이 모델에서 사용한 splitter: `RecursiveCharacterTextSplitter` 인스턴스의 `split_documents()` 함수 in `LangChain`
  - [랭체인 노트](https://wikidocs.net/233999)  
  - [코드 빠르게 읽기](https://lordofkangs.tistory.com/297)  


In [ ]:
test_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)

In [ ]:
docs = test_splitter.split_documents(data)

In [ ]:
# print(len(docs))

168


In [ ]:
# docs[16]

Document(page_content='parameterization to obtain the ﬁnal inference model. One\nis to train multiple identical models with different train-\ning data, and then average the weights of multiple trained\nmodels. The other is to perform a weighted average of the\nweights of models at different iteration number. Module-\nlevel re-parameterization is a more popular research issue\nrecently. This type of method splits a module into multi-\nple identical or different module branches during training', metadata={'source': 'pdfs/yolov7paper.pdf', 'page': 1})

## Step 06: Download the Embeddings From the Hugging Face
- Hugging Face: model, dataset 등을 얻을 수 있는 대형 AI 커뮤니티.  
  임베딩에 사용한 모델: `model_name='sentence-transformers/all-MiniLM-L6-v2'`  
- <mark>Embedding</mark> in ML: 각 객체에 대한 의미 있는 데이터를 캡처하기 위해 머신러닝 모델이 벡터를 생성하는 작업. 임베딩을 쓰면 머신러닝 모델이 _유사한 개체를 찾을 수 있다_. 임베딩은 머신러닝 모델과 시맨틱(유사성) 검색 알고리즘(semantic search algorithms)에서 사용하도록 설계되었다.  
  
  [Semantic Search란?](https://www.elastic.co/kr/what-is/semantic-search)  
  
  [NLP도 필요한 이유](https://www.elastic.co/kr/what-is/natural-language-processing)  
  
  밑의 링크는 그냥 아이디어, 컨셉 확장판~ 잠시 급발진하고 갑니당  
  [한발 더 나아가는, 어텐션 메커니즘](https://wikidocs.net/22893)  
  - [밑바닥버전](https://glee1228.tistory.com/3)  
  - [어텐션&트랜스포머](https://engineering-ladder.tistory.com/73)
  - [AI 경량화](https://engineering-ladder.tistory.com/79?category=826971)  
  - [Meta-Learning](https://engineering-ladder.tistory.com/95)  
  - [GPU!](https://engineering-ladder.tistory.com/108)  
  - [RLHF(ChatGPT 학습 방법) 분석](https://engineering-ladder.tistory.com/127)
  - [Approach: Continual Learning, Transfer Learning, Meta-Learning](https://engineering-ladder.tistory.com/94)  
  - [(뜬금없이) 이차 전지! Value Chain](https://engineering-ladder.tistory.com/128)  
  - [텐서플로가 가벼워진다면?](https://engineering-ladder.tistory.com/109)  
  - [AI 연산에 최적화된 칩, NPU](https://engineering-ladder.tistory.com/97)  
    - PLUS: Spiking Neural Network (SNN)은 뇌의 시냅스에서 전기 신호가 오고가는 것에서 착안한 방식으로 일정 기간동안 작은 전기 신호들이 모이면 하나의 큰 스파이크를 형성해서 출력되는 방식으로 동작한다. 이처럼 AI가 인간을 모방하는 것이라면, 연산하는 방식도 모방해보자는 Neuromorphic Chip이 연구되고 있다.  
  - [하이퍼파라미터 튜닝](https://engineering-ladder.tistory.com/75)  
  - [강화학습 DQN](https://engineering-ladder.tistory.com/68)  
  - [연합학습](https://engineering-ladder.tistory.com/66)  
  - [OpenAI Gym](https://engineering-ladder.tistory.com/61)  
  - [Policy Gradient Algorithm](https://engineering-ladder.tistory.com/58)
  - [Challenges!](https://engineering-ladder.tistory.com/57)  
  - [AI 기초](https://engineering-ladder.tistory.com/38)  
  - [DL이란?](https://engineering-ladder.tistory.com/39)  
  - [RNN vs LSTM vs GRU](https://engineering-ladder.tistory.com/41)  
  - [완전 좋은 블로그](https://hengbokhan.tistory.com/category/What%20is%20%E2%96%A1%3F)  
  - [벡터 데이터베이스를 사용하면 어떤 이점이 있을까요?](https://www.cloudflare.com/ko-kr/learning/ai/what-is-vector-database/)  

  사용: LLM의 경우 임베딩이 한 단계 더 진행된다. 단어 자체뿐만 아니라 모든 단어의 _문맥_ 이 임베딩된다. 전체 문장, 단락, 글 전체의 의미를 검색하고 분석할 수 있다. 이 작업에는 상당한 연산 능력이 필요하지만, 쿼리(query)에 대한 컨텍스트(context)를 임베딩으로 저장하여 향후 쿼리를 위한 시간과 연산 능력을 절약할 수 있다.  
   
- <mark>Vector</mark> in ML: 텍스트의 의미를 담고 있는 일련의 숫자. 각 숫자는 지정된 차원(dimension)을 따라 개체의 위치를 나타낸다.
_머신러닝에서 벡터를 사용하면 유사한 개체를 검색할 수 있다_. 벡터 검색 알고리즘은 벡터 데이터베이스에서 서로 가까운 두 개의 벡터를 찾기만 하면 된다. 대표적인 예시: 위도와 경도

- <mark>VectorDB</mark>: 수학적 표현으로 저장된 데이터의 모음. 벡터 데이터베이스를 사용하면 머신러닝 모델이 이전 입력을 더 쉽게 _기억_ 할 수 있다. 따라서 머신러닝은 강력한 검색, 추천, 텍스트 생성 사용 등에도 사용할 수 있다. _정확한 일치 대신 유사성(semantic) 메트릭을 기반으로_ 데이터를 식별할 수 있으므로 컴퓨터 모델이 맥락에 맞게 데이터를 이해할 수 있다.

- <mark>Query</mark> in ML: DB에 Client(사용자)가 원하는 특정 데이터를 보여달라는 _요청_. 어떠한 키워드를 검색/요청 시 필요한 옵션을 전달할 때 사용.
but, Parameter는 특정 id, 이름을 가지고 조회할 때 사용. Query가 좀 더 조건이 복잡하고, 사용자가 원하는 정제된 결과물을 얻을 수 있는 방법. 이러한 질의를 위한 특정한 구조의 프로그램 언어를 사용하기도 하며 대표적으로 SQL(Structured Query Language)이 있다.  
[쿼리 query란 무엇일까? Tistory Blog](https://kwonkai.tistory.com/40)  
    
Path parameter(따로 정제되지 않은 데이터를 불러올 때): 원하는 조건의 데이터 or 하나의 데이터에 대한 정보를 받아올 때 유용.  
  
Query string: 필터링, 정렬, 검색 등에서 적절하게 사용. URL 끝에 물음표로 시작하는 문자열이다. 쿼리 파라미터라고도 부른다. SQL의 명령어를 이르는 쿼리와는 다르다. 검색 사이트에 `?query=XXX` 라든지, `?redirect=URL` 이런 것이 쿼리 문자열이다. 다만 사용할 수 있는 데이터에 제한이 있다. 무조건 ASCII로만 전송해야 하며, 그나마도 숫자, 로마자, 일부 특수문자만 가능하다. 이 이외의 문자를 보내려면 _퍼센트 인코딩_ 을 거쳐야 한다.

Parsing: 웹페이지에서 원하는 데이터를 추출하여 가공하기 쉬운 상태로 바꾸는 것.  
웹페이지에 돌아다니는 데이터는 리스트, 딕셔너리 같은 자료구조와는 달라서 사용자 마음대로 접근하고 응용하기 쉽지 않다. 이런 데이터들을 다루기 쉬운 형태로 바꿔주는 함수나 프로그램을 파서(parser)라고 하며, 이 과정을 파싱(parsing)이라고 한다. 웹 크롤링을 할 때 필연적으로 만나게 되며, Python에서는 `BeautifulSoup` 라이브러리를 사용하여 html 문서를 파싱한다.

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# query_result = embeddings.embed_query("Hello World") # Hello World 문자열을 embedding
# print("Length", len(query_result))  # embedding 길이 (차원. dimension. embedding model에 따라 결정되는 듯하다. 일단은.)

Length 384


## Step 07: Initializing the Pinecone -> init is no longer...

In [ ]:
# 세션이 다시 시작되면 다시 실행해야 하는 것 같다!
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', '06f14569-b4f5-4445-a214-79f6d97a54f2')  # find at app.pinecone.io
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV', 'gcp-starter')  # next to api key in console

In [ ]:
# Old code (2023.09.07)
# initialize pinecone
# pinecone.init(
#     api_key=PINECONE_API_KEY,
#     environment=PINECONE_API_ENV
# )
# index_name = "pdfreader2"  # put in the name of your pinecone index here

In [ ]:
''' New code (2023.09.07 -> 2024.04.06)
AttributeError: init is no longer a top-level attribute of the pinecone package.
Please create an instance of the Pinecone class instead.
Example:
'''

import os
from pinecone import Pinecone, PodSpec

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = 'pdfreader2'
# api_key = PINECONE_API_KEY

# pc.create_index(
#     name='pdfreader2', # my index name
#     dimension=384,  # my dimension
#     metric='cosine',  # my metric
#     spec=PodSpec(
#         environment = 'gcp-starter',
#         pod_type='starter',
#         pods=1
#     )
# )

if 'pdfreader2' not in pc.list_indexes().names():
    pc.create_index(
        name='pdfreader2',
        dimension=384,
        metric='cosine',
        spec=PodSpec(
            environment = 'gcp-starter',
            pod_type='starter',
            pods=1
            )
    )

In [ ]:
# Delete an index
# from pinecone import Pinecone, PodSpec
# pc = Pinecone(api_key="06f14569-b4f5-4445-a214-79f6d97a54f2")
# pc.delete_index("pdfreader")  # Resource pdfreader not found...
# pc.delete_index("pdfreader2")

In [ ]:
# If you are using Pinecone serverless and getting the error
# "AttributeError: module 'pinecone' has no attribute 'init',
# first check that you are using the latest version of the Python client.

# You can check the version of the client by running:
# !pip list pinecone-client
# pinecone-client: v3.2.2
# langchain-pinecone: none

## Step 08: Create Embeddings for each of the Text Chunk
이거 전에 Upsert vectors를 해야하나?  
`docsearch` 변수는 `Pinecone`의 `langchain.vectorstores` 모듈에 있는 `from_texts()` 함수를 이용하여 chunks를 vectorstore로 만들기.  
`docs` 변수에 있는 PDF chunks를 `t.page_content` 리스트 내포.  
또한 `embeddings`와 `index_name`이 필요하다.  

In [ ]:
# Upsert vectors
# 이제 인덱스를 만들었으므로 샘플 벡터를 2개의 고유한 네임스페이스에 삽입하세요 .

# 네임스페이스를 사용하면 단일 인덱스 내에서 벡터를 분할할 수 있습니다.
# 선택 사항이지만 네임스페이스별로 필터링할 수 있는 쿼리 속도를 높이고
# 다중 테넌트 요구 사항을 준수하기 위한 모범 사례입니다.

# index = pc.Index("pdfreader2")

In [ ]:
# !pip install -U langchain-pinecone

In [ ]:
import os
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY
# PineconeConfigurationError: You haven't specified an Api-Key.

# AttributeError: type object 'Pinecone' has no attribute 'from_texts'
# > use vectorstore?
from langchain.vectorstores import Pinecone

docsearch = Pinecone.from_texts([t.page_content for t in docs], embeddings, index_name=index_name)

In [ ]:
# Pinecone has no attribute 'from_texts' errors occur when using an outdated version of LangChain.
# Previously, the Python classes for both LangChain and Pinecone had objects named Pinecone,
# but this is no longer an issue in the latest LangChain version.


## If you already have an index, you can load it like this

In [ ]:
# docsearch = Pinecone.from_existing_index(index_name, embeddings)

AttributeError: type object 'Pinecone' has no attribute 'from_existing_index'

## Step 09: Similarity Search
드디어! query를 넣어본다.  
LangChain의 `similarity_search()` 함수를 사용하여 PDF 내에서 query와 유사한 결과를 뽑아온다.  

In [ ]:
# query = "YOLOv7 outperforms which models"

In [ ]:
# docs = docsearch.similarity_search(query)
# docs

[Document(page_content='From the results we see that if compared with YOLOv4,\nYOLOv7 has 75% less parameters, 36% less computation,\nand brings 1.5% higher AP. If compared with state-of-the-\nart YOLOR-CSP, YOLOv7 has 43% fewer parameters, 15%\nless computation, and 0.4% higher AP. In the performance\nof tiny model, compared with YOLOv4-tiny-31, YOLOv7-\ntiny reduces the number of parameters by 39% and the\namount of computation by 49%, but maintains the same AP.\nOn the cloud GPU model, our model can still have a higher'),
 Document(page_content='From the results we see that if compared with YOLOv4,\nYOLOv7 has 75% less parameters, 36% less computation,\nand brings 1.5% higher AP. If compared with state-of-the-\nart YOLOR-CSP, YOLOv7 has 43% fewer parameters, 15%\nless computation, and 0.4% higher AP. In the performance\nof tiny model, compared with YOLOv4-tiny-31, YOLOv7-\ntiny reduces the number of parameters by 39% and the\namount of computation by 49%, but maintains the same AP.\

## Step 10: Creating a Llama2 Model Wrapper
Llama2 Model(`Llama-2-7b-chat-hf`)을 Hugging Face에서 가져와서,  
`AutoTokenizer.from_pretrained()` 실행하여 `tokenizer` 생성.  
LLM을 토큰화하면 좋다고 했는데!  
`model` 객체를 만들기 위해 `AutoModelForCausalLM.from_pretrained()` 이것도 필요하다.  
요 함수부터는 매개변수가 뭔지 몰루... 나중에 알아보자.  
최적화를 위해 필요한 듯하다.  

`pipeline()` 함수로 `pipe` 객체를 만든다.  
방금 만든 `pipe`를 이용, `HuggingFacePipeline()` 함수로 `llm` 객체를 만든다.  


In [ ]:
from huggingface_hub import notebook_login
import torch

In [ ]:
notebook_login()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf", token=True)  # FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
# ImportError: Using `bitsandbytes` 8-bit quantization requires Accelerate: `pip install accelerate`
# and the latest version of bitsandbytes: `pip install -i https://pypi.org/simple/ bitsandbytes`
# > But its for CPU running: change the environment to GPU. and the issue will go away anyway.
# > That is colab CPU and GPU uses different transformer version.
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             token=True,  # FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
                                             # Please use `token` instead.
                                             load_in_8bit=True)  # The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions.
                                             # Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
pipe = pipeline("text-generation",
                model=model,
                tokenizer=tokenizer,
                torch_dtype=torch.bfloat16,
                device_map='auto',
                max_new_tokens=512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id)

In [ ]:
llm = HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0.1})

## Step 11: Create a Prompt Template
OpenAI의 Playground와 유사한 작업.  
`SYSTEM_PROMPT` 만들어서 PDF 맥락을 기반으로 하라는 명령과, 환각 현상을 대비한 문구 작성.  
`PromptTemplate()`함수를 이용하여 만든 `prompt`는 이 프로그램이 실행될 때마다 나오는 문구를 설정한다.  
`RetrievalQA.from_chain_type()`함수를 이용하여 만든 `qa_chain`은 LLM과 chain을 형성하여 User와 QNA를 할 수 있도록 한다.  
`result`는 `qa_chain()` 안에 문자열을 집어넣어 query 생성.  
`result['result']`로 결과 확인.  

마지막으로 while True문 만들어서  
사용자가 exit 입력할 때까지 `input()`이용하여 `user_input`에 query를 입력받고,  
Answer로 `result['result']` 출력하면 끄읏~~


In [ ]:
SYSTEM_PROMPT = """Use the following pieces of context to answer the question at the end,
If you don't know the answer, just say that you don't know, don't try to make up an answer."""

In [ ]:
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<<SYS>>\n\n"

In [ ]:
SYSTEM_PROMPT = B_SYS + SYSTEM_PROMPT + E_SYS

In [ ]:
instruction = """
{context}

Question: {question}
"""

In [ ]:
template = B_INST + SYSTEM_PROMPT + instruction + E_INST

In [ ]:
# template

"[INST]<<SYS>>\nUse the following pieces of context to answer the question at the end,\nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n<<SYS>>\n\n\n{context}\n\nQuestion: {question}\n[/INST]"

In [ ]:
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt})

In [ ]:
# result = qa_chain("YOLOv7 is used for")
# LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [ ]:
# result['result']

"[INST]<<SYS>>\nUse the following pieces of context to answer the question at the end,\nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n<<SYS>>\n\n\ntion, in terms of the amount of parameters and computation,\nYOLOv7-X reduces 22% of parameters and 8% of compu-\ntation compared to YOLOv5-X (r6.1), but improves AP by\n2.2%.\n7\n\ntion, in terms of the amount of parameters and computation,\nYOLOv7-X reduces 22% of parameters and 8% of compu-\ntation compared to YOLOv5-X (r6.1), but improves AP by\n2.2%.\n7\n\nQuestion: YOLOv7 is used for\n[/INST]  Based on the context provided, YOLOv7 is used for object detection."

In [ ]:
while True:
  user_input=input(f"prompt:")
  if user_input=='exit':
    print('Exiting')
    sys.exit()
  if user_input=='':
    continue
  result=qa_chain({'query': user_input})
  print(f"Answer:{result['result']}")

prompt:give main keywords in "yolov7paper.pdf" and make a table.
Answer:[INST]<<SYS>>
Use the following pieces of context to answer the question at the end,
If you don't know the answer, just say that you don't know, don't try to make up an answer.
<<SYS>>


YOLOv5-X (r6.1) [23] 86.7M 205.7G 640 83 - / 50.7% - - - - -
YOLOR-CSP [81] 52.9M 120.4G 640 106 51.1% / 50.8% 69.6% 55.7% 31.7% 55.3% 64.7%
YOLOR-CSP-X [81] 96.9M 226.8G 640 87 53.0% / 52.7% 71.4% 57.9% 33.7% 57.1% 66.8%
YOLOv7-tiny-SiLU 6.2M 13.8G 640 286 38.7% / 38.7% 56.7% 41.7% 18.8% 42.4% 51.9%
YOLOv7 36.9M 104.7G 640 161 51.4% / 51.2% 69.7% 55.9% 31.8% 55.5% 65.0%
YOLOv7-X 71.3M 189.9G 640 114 53.1% / 52.9% 71.2% 57.8% 33.8% 57.1% 67.4%

YOLOv5-X (r6.1) [23] 86.7M 205.7G 640 83 - / 50.7% - - - - -
YOLOR-CSP [81] 52.9M 120.4G 640 106 51.1% / 50.8% 69.6% 55.7% 31.7% 55.3% 64.7%
YOLOR-CSP-X [81] 96.9M 226.8G 640 87 53.0% / 52.7% 71.4% 57.9% 33.7% 57.1% 66.8%
YOLOv7-tiny-SiLU 6.2M 13.8G 640 286 38.7% / 38.7% 56.7% 41.7% 18.8% 42

SystemExit: 

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
